<a href="https://colab.research.google.com/github/RogueOne-22/Lab-1---Introduccion-IA/blob/main/1er_punto_Lab_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import heapq

class Puzzle:
    def __init__(self, board, moves=0, parent=None):
        self.board = board
        self.moves = moves
        self.parent = parent
        self.priority = moves + self.manhattan_distance()

    def __lt__(self, other):
        return self.priority < other.priority

    def manhattan_distance(self):
        """Calcula la distancia Manhattan (heurística)"""
        distance = 0
        goal = {1:(0,0), 2:(0,1), 3:(0,2), 4:(1,0), 5:(1,1),
                6:(1,2), 7:(2,0), 8:(2,1), 0:(2,2)}

        for i, tile in enumerate(self.board):
            if tile != 0:
                current_row, current_col = i // 3, i % 3
                goal_row, goal_col = goal[tile]
                distance += abs(current_row - goal_row) + abs(current_col - goal_col)

        return distance

    def is_solved(self):
        """Verifica si el puzzle está resuelto"""
        return self.board == [1, 2, 3, 4, 5, 6, 7, 8, 0]

    def get_neighbors(self):
        """Genera estados vecinos moviendo el espacio vacío"""
        neighbors = []
        blank = self.board.index(0)
        row, col = blank // 3, blank % 3

        # Movimientos: arriba, abajo, izquierda, derecha
        for dr, dc in [(-1,0), (1,0), (0,-1), (0,1)]:
            new_row, new_col = row + dr, col + dc

            # Verificar límites del tablero
            if 0 <= new_row < 3 and 0 <= new_col < 3:
                new_pos = new_row * 3 + new_col
                new_board = self.board.copy()

                # Intercambiar espacio vacío con la ficha
                new_board[blank], new_board[new_pos] = new_board[new_pos], new_board[blank]
                neighbors.append(Puzzle(new_board, self.moves + 1, self))

        return neighbors

    def print_board(self):
        """Imprime el tablero de forma visual"""
        for i in range(0, 9, 3):
            row = [str(self.board[i+j]) if self.board[i+j] != 0 else ' '
                   for j in range(3)]
            print(f"| {' | '.join(row)} |")
        print()

def is_solvable(board):
    """Verifica si el puzzle es solvable (número par de inversiones)"""
    inversions = 0
    flat = [x for x in board if x != 0]

    for i in range(len(flat)):
        for j in range(i + 1, len(flat)):
            if flat[i] > flat[j]:
                inversions += 1

    return inversions % 2 == 0

def solve_puzzle(initial_board):
    """Resuelve el puzzle usando A* con heurística Manhattan"""

    # Verificar si es solvable
    if not is_solvable(initial_board):
        return None, "El puzzle no tiene solución"

    initial = Puzzle(initial_board)

    # Si ya está resuelto
    if initial.is_solved():
        return [initial], "Ya está resuelto"

    # A* con heap
    heap = [initial]
    visited = set()

    while heap:
        current = heapq.heappop(heap)

        # Convertir a tupla para poder usar en set
        state_key = tuple(current.board)
        if state_key in visited:
            continue

        visited.add(state_key)

        # Generar vecinos
        for neighbor in current.get_neighbors():
            if neighbor.is_solved():
                # Reconstruir camino
                path = []
                temp = neighbor
                while temp:
                    path.append(temp)
                    temp = temp.parent
                return path[::-1], f"Resuelto en {neighbor.moves} movimientos"

            neighbor_key = tuple(neighbor.board)
            if neighbor_key not in visited:
                heapq.heappush(heap, neighbor)

    return None, "No se encontró solución"

def print_solution(solution, message):
    """Imprime la solución paso a paso"""
    print(f"\n{message}\n" + "="*40)

    if solution:
        for i, state in enumerate(solution):
            print(f"\nPaso {i} (Heurística: {state.manhattan_distance()}):")
            state.print_board()
    else:
        print("No hay solución disponible")

def ejemplo_personalizado():
    """Permite crear un puzzle personalizado"""
    print("\n🧩 CREA TU PUZZLE:")
    print("Ingresa 9 números (0-8) separados por espacios")
    print("Ejemplo: 1 2 3 4 0 6 7 5 8")

    try:
        user_input = input("\nTu puzzle: ").strip()
        board = [int(x) for x in user_input.split()]

        # Validar entrada
        if len(board) != 9 or set(board) != set(range(9)):
            print("❌ Error: Debes ingresar los números del 0 al 8")
            return

        print("\nTu puzzle:")
        Puzzle(board).print_board()

        solution, message = solve_puzzle(board)
        print_solution(solution, message)

    except ValueError:
        print("❌ Error: Solo números separados por espacios")

# ANÁLISIS DE LA HEURÍSTICA
def analizar_heuristica(board):
    """Muestra cómo funciona la heurística Manhattan"""
    puzzle = Puzzle(board)

    print("🔍 ANÁLISIS DE HEURÍSTICA MANHATTAN:")
    print("="*40)
    puzzle.print_board()

    goal_positions = {1:(0,0), 2:(0,1), 3:(0,2), 4:(1,0), 5:(1,1),
                     6:(1,2), 7:(2,0), 8:(2,1), 0:(2,2)}

    total_distance = 0
    print("Distancias por ficha:")

    for i, tile in enumerate(board):
        if tile != 0:
            current_row, current_col = i // 3, i % 3
            goal_row, goal_col = goal_positions[tile]
            distance = abs(current_row - goal_row) + abs(current_col - goal_col)
            total_distance += distance

            print(f"Ficha {tile}: posición ({current_row},{current_col}) → objetivo ({goal_row},{goal_col}) = distancia {distance}")

    print(f"\n📊 Heurística total: {total_distance}")
    print(f"🎯 Esto significa que necesitamos mínimo {total_distance} movimientos")

# PROGRAMA PRINCIPAL
if __name__ == "__main__":
    print("🧩 SOLUCIONADOR 8-PUZZLE CON MANHATTAN")
    print("="*50)

    # Ejecutar ejemplos
    ejemplo_personalizado()

    # Análisis de heurística
    print("\n" + "="*50)
    analizar_heuristica([1, 2, 3, 0, 4, 6, 7, 5, 8])

    # Descomenta para probar tu propio puzzle:
    # ejemplo_personalizado()

🧩 SOLUCIONADOR 8-PUZZLE CON MANHATTAN

🧩 CREA TU PUZZLE:
Ingresa 9 números (0-8) separados por espacios
Ejemplo: 1 2 3 4 0 6 7 5 8

Tu puzzle: 5 4 3 1 2 8 0 7 6

Tu puzzle:
| 5 | 4 | 3 |
| 1 | 2 | 8 |
|   | 7 | 6 |


Resuelto en 20 movimientos

Paso 0 (Heurística: 10):
| 5 | 4 | 3 |
| 1 | 2 | 8 |
|   | 7 | 6 |


Paso 1 (Heurística: 11):
| 5 | 4 | 3 |
|   | 2 | 8 |
| 1 | 7 | 6 |


Paso 2 (Heurística: 10):
|   | 4 | 3 |
| 5 | 2 | 8 |
| 1 | 7 | 6 |


Paso 3 (Heurística: 9):
| 4 |   | 3 |
| 5 | 2 | 8 |
| 1 | 7 | 6 |


Paso 4 (Heurística: 8):
| 4 | 2 | 3 |
| 5 |   | 8 |
| 1 | 7 | 6 |


Paso 5 (Heurística: 7):
| 4 | 2 | 3 |
| 5 | 8 |   |
| 1 | 7 | 6 |


Paso 6 (Heurística: 8):
| 4 | 2 |   |
| 5 | 8 | 3 |
| 1 | 7 | 6 |


Paso 7 (Heurística: 9):
| 4 |   | 2 |
| 5 | 8 | 3 |
| 1 | 7 | 6 |


Paso 8 (Heurística: 10):
|   | 4 | 2 |
| 5 | 8 | 3 |
| 1 | 7 | 6 |


Paso 9 (Heurística: 11):
| 5 | 4 | 2 |
|   | 8 | 3 |
| 1 | 7 | 6 |


Paso 10 (Heurística: 10):
| 5 | 4 | 2 |
| 1 | 8 | 3 |
|   | 7 | 6 |


